# OptiGuide Example




Here we give a simple example, as designed and illustrated in the [OptiGuide paper](https://arxiv.org/abs/2307.03875).
While the original paper is designed specifically for supply chain optimization, the general framework can be easily adapted to other applications with coding capacity.




## OptiGuide for Supply Chain Optimization: System Design Overview

The original system design for OptiGuide, tailored for supply chain optimization, is presented below.

The collaboration among three agents -- Coder, Safeguard, and Interpreter -- lies at the core of this system. They leverage a set of external tools and a large language model (LLM) to address users' questions related to supply chain applications. For a comprehensive understanding of the design and data flow, detailed information can be found in the original [paper](https://arxiv.org/abs/2307.03875).


![optiguide system](https://www.beibinli.com/docs/optiguide/optiguide_system.png)


## New Implementation



![](new_design.png)

Advantages of this multi-agent design with autogen:
- Collaborative Problem Solving: The collaboration among the user proxy agent and the assistant agents fosters a cooperative problem-solving environment. The agents can share information and knowledge, allowing them to complement each other's abilities and collectively arrive at better solutions. On the other hand, the Safeguard acts as a virtual adversarial checker, which can perform another safety check pass on the generated code.

- Modularity: The division of tasks into separate agents promotes modularity in the system. Each agent can be developed, tested, and maintained independently, simplifying the overall development process and facilitating code management.

- Memory Management: The OptiGuide agent's role in maintaining memory related to user interactions is crucial. The memory retention allows the agents to have context about a user's prior questions, making the decision-making process more informed and context-aware.



In [ ]:
!pip install gurobipy -q
!pip install eventlet -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 85.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 11.5 MB/s eta 0:00:00


In [ ]:
!pip install tiktoken -q
!pip install cohere -q
!pip install openai -q
!pip install flaml -q
!pip install optiguide -q
!pip install flaml[openai] -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 34.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires openai, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.6 MB/s eta 0:00:00
   

In [ ]:
!pip install pyautogen -q
!pip install "flaml[openai]" -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.7/122.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.2/220.2 kB 10.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyautogen 0.2.1 requires openai<1.3,~=1.2, but you have openai 0.27.8 which is incompatible.


In [ ]:
# test Gurobi installation
import gurobipy as gp
from gurobipy import GRB
from eventlet.timeout import Timeout

# import auxillary packages
import requests  # for loading the example source code
import openai

# import flaml and autogen
from flaml import autogen
from flaml.autogen.agentchat import Agent, UserProxyAgent
from optiguide.optiguide import OptiGuideAgent


In [ ]:
# Install Required Packages
# %pip install optiguide
# import os
openai.api_key = 'your api key'

In [ ]:
autogen.oai.ChatCompletion.start_logging()

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": {
            "gpt-4",
            # "gpt4",
            # "gpt-4-32k",
            # "gpt-4-32k-0314",
            # "gpt-3.5-turbo",
            # "gpt-3.5-turbo-16k",
            # "gpt-3.5-turbo-0301",
            # "chatgpt-35-turbo-0301",
            # "gpt-35-turbo-v0301",
        }
    }
)

Now, let's import the source code (loading from URL) and also some training examples (defined as string blow).

In [ ]:
# Get the source code of our coffee example
code_url = "https://raw.githubusercontent.com/jiuyuanx/LLM-Gurobi/main/model1.py"
response  = requests.get(code_url)


# Check if the request was successful
if response.status_code == 200:
    # Get the text content from the response
    code = response.text
else:
    raise RuntimeError("Failed to retrieve the file.")
# code = open(code_url, "r").read() # for local files


# show the first head and tail of the source code
print("\n".join(code.split("\n")[:10]))
print(".\n" * 3)
print("\n".join(code.split("\n")[-10:]))

from gurobipy import Model, GRB, quicksum
# Example data
C = 30000  # Total consulting payment
D = 7000   # Initial credit card debt
r1 = 0.1595 / 12  # Monthly APR of Card 1
r2 = 0.029 / 12   # Monthly APR of Card 2
r3 = 0.002        # Transfer fee rate
B = 70000     # Annual base salary
alpha = 80000     # Tax threshold parameter

.
.
.

if model.status == GRB.OPTIMAL:
    print("Optimal Wealth:", model.objVal)
    for t in range(6):
        print(f"Month {t+1}: Payment {round(P1[t].x, 2)}, Payment2 {round(P2[t].x, 2)} Transfer {round(T[t].x,2)}, Salary {round(S[t].x,2)}, Balance1 {round(balance1[t].x,2)}, Balance2 {round(balance2[t].x,2)}")

if m.status == GRB.OPTIMAL:
    print(f'Optimal cost: {m.objVal}')
else:
    print("Not solved to optimality. Optimization status:", m.status)



In [ ]:
# In-context learning examples.
example_qa = """

Question: What if there's APR of Card 2 goes to 0.05?
Answer Code:
```python
r2 = 0.05   # APR of Card 2
```
"""

In [ ]:
# # In-context learning examples.
# example_qa = """
# ----------
# Question: Why is it not recommended to use just one supplier for roastery 2?
# Answer Code:
# ```python
# z = m.addVars(suppliers, vtype=GRB.BINARY, name="z")
# m.addConstr(sum(z[s] for s in suppliers) <= 1, "_")
# for s in suppliers:
#     m.addConstr(x[s,'roastery2'] <= capacity_in_supplier[s] * z[s], "_")
# ```

# ----------
# Question: What if there's a 13% jump in the demand for light coffee at cafe1?
# Answer Code:
# ```python
# light_coffee_needed_for_cafe["cafe1"] = light_coffee_needed_for_cafe["cafe1"] * (1 + 13/100)
# ```
# """

Now, let's create an OptiGuide agent and also a user.

For the OptiGuide agent, we only allow "debug_times" to be 1, which means it can debug its answer once if it encountered errors.

In [ ]:
%%capture
agent = OptiGuideAgent(
    name="Personal Finance Optimization",
    source_code=code,
    debug_times=1,
    example_qa = example_qa,
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    }
)

user = UserProxyAgent(
    "user", max_consecutive_auto_reply=0,
    human_input_mode="NEVER", code_execution_config=False
)

Now, let's create a user's question.

In [ ]:
#Q1 - on the ppt, parameter change
user.initiate_chat(agent, message="What if my annual salary is 22000?")

user (to Personal Finance Optimization):

What if my annual salary is 22000?

--------------------------------------------------------------------------------
Personal Finance Optimization (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to Personal Finance Optimization):

```python
B = 22000     # Annual base salary
```

--------------------------------------------------------------------------------
Personal Finance Optimization (to safeguard):


--- Code ---
B = 22000     # Annual base salary

--- One-Word Answer: SAFE or DANGER ---


--------------------------------------------------------------------------------
safeguard (to Personal Finance Optimization):

SAFE

--------------------------------------------------------------------------------
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count:

In [ ]:
#Q2 - on the ppt, constraint change
user.initiate_chat(agent, message="What if the first two months payment cannot exceed 5000?")

user (to Personal Finance Optimization):

What if the first two months payment cannot exceed 5000?

--------------------------------------------------------------------------------
Personal Finance Optimization (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to Personal Finance Optimization):

```python
# Add constraints for the first two months
model.addConstr(P1[0] + P2[0] <= 5000)
model.addConstr(P1[1] + P2[1] <= 5000)
```

--------------------------------------------------------------------------------
Personal Finance Optimization (to safeguard):


--- Code ---
# Add constraints for the first two months
model.addConstr(P1[0] + P2[0] <= 5000)
model.addConstr(P1[1] + P2[1] <= 5000)

--- One-Word Answer: SAFE or DANGER ---


--------------------------------------------------------------------------------
safeguard (to Personal Finance Optimization):

SAFE

----------------------------------------------------------

In [ ]:
#Q3 - on the ppt, new situation
user.initiate_chat(agent, message="There is another option to invest in a new 80% annual return rate project")

user (to Personal Finance Optimization):

There is another option to invest in a new 80% annual return rate project

--------------------------------------------------------------------------------
Personal Finance Optimization (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to Personal Finance Optimization):

```python
# Add investment option
I = model.addVars(6, name="I")        # Investment to new project
r_project = 0.80  # Annual return rate of the project

# Adjust objective
interest = quicksum(I[t]*((1 + r_project)**(6-t)) for t in range(6))
wealth = B*2 + quicksum(S[t] - P1[t] - P2[t] - I[t] for t in range(6)) - total_tax + interest
model.setObjective(wealth, GRB.MAXIMIZE)

# Add constraint to ensure only surplus money is used for investment
[model.addConstr(I[t] - (S[t]+quicksum([S[i]-P1[i]-P2[i]-I[i] for i in range(t)])) <= 0) for t in range(6)]
```

--------------------------------------------------------

In [ ]:
#Q4 - on the ppt, stochastic optimization
user.initiate_chat(agent, message="There is 30% chance of me being fired after 4 months, perform stochastic optimization")

user (to Personal Finance Optimization):

There is 30% chance of me being fired after 4 months, perform stochastic optimization

--------------------------------------------------------------------------------
Personal Finance Optimization (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to Personal Finance Optimization):

```python
from gurobipy import GRB

# Assume that if you are fired, you don't get any salary for the remaining months
# Thus, let's add another constraint

# Add a binary variable for each month to represent whether we are fired or not
F = model.addVars(6, vtype=GRB.BINARY, name="F")

# Define a parameter to indicate the probability of being fired after 4 months
fire_prob = 0.3

# Define wealth for both scenarios
wealth_fired = B*2*fire_prob + quicksum((1-F[t])*(S[t] - P1[t] - P2[t]) + F[t]*(B/12) for t in range(6)) - total_tax
wealth_not_fired = B*2*(1-fire_prob) + quicksum((1-F[t])*(S[t] - P1[t] 

In [ ]:
#Q5,
user.initiate_chat(agent, message="What if the initial debt goes to 20000?")

user (to Personal Finance Optimization):

What if the initial debt goes to 20000?

--------------------------------------------------------------------------------
Personal Finance Optimization (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to Personal Finance Optimization):

```python
D = 20000  # Update Initial credit card debt

# Update Transfer Constraint
for t in range(6):
    model.remove(model.getConstrByName(f"C{t+8}"))
    model.addConstr(T[t] <= D - quicksum(P1[k] for k in range(t)))
```

--------------------------------------------------------------------------------
Personal Finance Optimization (to safeguard):


--- Code ---
D = 20000  # Update Initial credit card debt

# Update Transfer Constraint
for t in range(6):
    model.remove(model.getConstrByName(f"C{t+8}"))
    model.addConstr(T[t] <= D - quicksum(P1[k] for k in range(t)))

--- One-Word Answer: SAFE or DANGER ---


---------------------------

In [ ]:
#Q6, parameter change
user.initiate_chat(agent, message="What if the tax threshold is 75000?")

user (to Personal Finance Optimization):

What if the tax threshold is 75000?

--------------------------------------------------------------------------------
Personal Finance Optimization (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to Personal Finance Optimization):

```python
# Modify the tax threshold parameter
alpha = 75000
```

--------------------------------------------------------------------------------
Personal Finance Optimization (to safeguard):


--- Code ---
# Modify the tax threshold parameter
alpha = 75000

--- One-Word Answer: SAFE or DANGER ---


--------------------------------------------------------------------------------
safeguard (to Personal Finance Optimization):

SAFE

--------------------------------------------------------------------------------
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction 

In [ ]:
#Q7, stochastic optimization, parameter change
user.initiate_chat(agent, message="My initial debt goes to 30000, but each month there is 40% chance I get 1000 dollars from my parents, \
which counts as part of total wealth, stochastic optimization")

user (to Personal Finance Optimization):

My initial debt goes to 30000, but each month there is 40% chance I get 1000 dollars from my parents, which counts as part of total wealth, stochastic optimization

--------------------------------------------------------------------------------
Personal Finance Optimization (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to Personal Finance Optimization):

```python
from gurobipy import GurobiError
import numpy as np

# Adjusting the initial debt
D = 30000  # Initial credit card debt
payments_from_parents = [1000 if np.random.rand() < 0.4 else 0 for _ in range(6)] # Simulating the stochastic payments in each month

# Adjust the model to account for the additional income
try:
    for t in range(6):
        S[t].lb += payments_from_parents[t]
    model.update()
except GurobiError as e:
    print('Error code ' + str(e.errno) + ": " + str(e))
```

------------------------------

In [ ]:
#Q8 multiple parameters change
user.initiate_chat(agent, message="What if the annual APR of credit card1 goes to 0.5, card2 goes to 0.2, \
and the last month balance does not have to be zero, but there is going to be 5000 dollars fine subtracted from the wealth?")

user (to Personal Finance Optimization):

What if the annual APR of credit card1 goes to 0.5, card2 goes to 0.2, and the last month balance does not have to be zero, but there is going to be 5000 dollars fine subtracted from the wealth?

--------------------------------------------------------------------------------
Personal Finance Optimization (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to Personal Finance Optimization):

```python
r1 = 0.5 / 12  # Monthly APR of Card 1
r2 = 0.2 / 12   # Monthly APR of Card 2
# Modify the end balance constraint
model.remove(model.getConstrByName('balance1[5]'))
model.remove(model.getConstrByName('balance2[5]'))
model.addConstr(balance1[5] + 5000 <= 0)
model.addConstr(balance2[5] + 5000 <= 0)

# Modify the objective function
total_tax = 0.1 * d1_1 + 0.28 * d2_1 + 0.1 * d1_2 + 0.28 * d2_2 
wealth = B*2 + quicksum(S[t] - P1[t] - P2[t] for t in range(6)) - total_tax - 5000*binary

In [ ]:
#Q9 new situation and constraint
user.initiate_chat(agent, message="What if there is an investment project of APR 80%, \
and also each month's balance of the credit card cannot exceed 5000?")

user (to Personal Finance Optimization):

What if there is an investment project of APR 80%, and also each month's balance of the credit card cannot exceed 5000?

--------------------------------------------------------------------------------
Personal Finance Optimization (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to Personal Finance Optimization):

```python
r_investment = 0.8 / 12  # Monthly APR of investment
I = model.addVars(6, name="I")   # Investment 

# Constraint: Sign up for this investment will automatically make a $5000 investment each month
for t in range(6):
    model.addConstr(I[t] == 5000)

# The profit from the investment is part of the wealth
wealth += quicksum(I[t] * (1 + r_investment) for t in range(6))

# Update Objective
model.setObjective(wealth, GRB.MAXIMIZE)

# Constraint: Don't overdraft. The balance of the credit card each month can not exceed 5000
for t in range(6):
    model.addCons

In [ ]:
#Q10
user.initiate_chat(agent, message="What if my parents give me 1000 dollars each month in six months we are considering and it could also be used to pay debt?")

user (to Personal Finance Optimization):

What if my parents give me 1000 dollars each month in six months we are considering and it could also be used to pay debt?

--------------------------------------------------------------------------------
Personal Finance Optimization (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to Personal Finance Optimization):

```python
# Parent's gift
G = 1000*6  # Total gift

# modify the wealth calculation
wealth = B*2 + quicksum(S[t] - P1[t] - P2[t] for t in range(6)) + G - total_tax
model.setObjective(wealth, GRB.MAXIMIZE)

# modify the payment constraint
for t in range(6):
    model.addConstr(P1[t] + P2[t] - (S[t]+quicksum([S[i]-P1[i]-P2[i] for i in range(t)]) + G/6) <= 0)
```

--------------------------------------------------------------------------------
Personal Finance Optimization (to safeguard):


--- Code ---
# Parent's gift
G = 1000*6  # Total gift

# modify the wealth 